[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prashantkul/simple-mcp-server/blob/claude/mcp-colab-customer-demo-011CUyKnnHZc4KxRNuBxQry3/mcp_customer_demo.ipynb)

# 🎓 Educational MCP Server Demo: Customer Management System

## Welcome to the Model Context Protocol (MCP) Tutorial!

This interactive notebook teaches you how MCP works by building a real customer management system.

**What you'll learn:**
- How MCP servers expose tools to AI assistants
- HTTP streaming with Server-Sent Events (SSE)
- Building practical MCP applications
- Database integration with MCP tools

**How to use this notebook:**
1. Run each cell in order from top to bottom
2. Read the explanations in markdown cells
3. Observe the output and MCP protocol messages
4. Experiment by modifying the test cells

Let's get started! 🚀

---


### 🔍 Testing with MCP Inspector

In this tutorial, you'll also learn how to test your MCP server using **MCP Inspector**, a web-based tool from Anthropic that lets you:
- Connect to any MCP server via HTTP/SSE
- Explore available tools interactively
- Send test requests and see responses
- Debug your MCP implementation

We'll automatically generate a public URL for your server so you can test it with MCP Inspector from anywhere!## 📚 What is MCP?

**Model Context Protocol (MCP)** is an open protocol that enables AI assistants like Claude to securely interact with external data sources and tools.

### Key Concepts:

1. **MCP Server**: A program that exposes tools, resources, or prompts to AI assistants
2. **MCP Client**: The AI assistant that connects to servers and uses their capabilities
3. **Tools**: Functions that the AI can call (like our customer management functions)
4. **Resources**: Data that the AI can read (like files or database records)
5. **Prompts**: Pre-defined templates the AI can use

### How MCP Works:

```
┌─────────────┐         ┌─────────────┐         ┌──────────────┐
│   Claude    │◄────────┤ MCP Protocol├────────►│  MCP Server  │
│ (AI Client) │  HTTP/  │   Messages  │  HTTP/  │ (Your Tools) │
│             │   SSE   │             │   SSE   │              │
└─────────────┘         └─────────────┘         └──────────────┘
                                                        │
                                                        ▼
                                                  ┌──────────┐
                                                  │ Database │
                                                  └──────────┘
```

### In This Demo:

We'll build an MCP server that exposes customer management tools. An AI assistant can then:
- List customers
- Get customer details
- Add new customers
- Update customer information
- Enable/disable customer accounts

All through the MCP protocol! 🎯

💡 **Learning Point**: MCP uses Server-Sent Events (SSE) for real-time streaming communication between the client and server.

---
## 🔧 Installation and Setup

First, let's install all the required packages for our MCP server.

In [ ]:
# Install required packages
!pip install flask flask-cors requests termcolor pyngrok -q

print("✅ All packages installed successfully!")
print("📦 Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)")


---
## 🗄️ Database Setup

Let's create a SQLite database to store customer information. SQLite is perfect for demos because it's:
- Lightweight (no separate server needed)
- File-based (stores data in a single file)
- Full-featured SQL database

💡 **Learning Point**: In Colab, files are stored in temporary storage. The database will persist during your session but will be deleted when the runtime is reset.

In [ ]:
import sqlite3
from datetime import datetime
import os

# Database file path
DB_PATH = '/content/customers.db'

def init_database():
    """
    Initialize the SQLite database with the customers table and sample data.
    """
    # Remove existing database if it exists (for clean start)
    if os.path.exists(DB_PATH):
        os.remove(DB_PATH)
        print("🗑️  Removed existing database")

    # Connect to database (creates file if it doesn't exist)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Create customers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT,
            phone TEXT,
            status TEXT DEFAULT 'active' CHECK(status IN ('active', 'disabled')),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # Create index on status for faster queries
    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_customer_status ON customers(status)
    ''')

    # Create index on email for lookups
    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_customer_email ON customers(email)
    ''')

    # Insert sample data
    sample_customers = [
        ('Alice Johnson', 'alice.johnson@email.com', '+1-555-0101', 'active'),
        ('Bob Smith', 'bob.smith@email.com', '+1-555-0102', 'active'),
        ('Carol White', 'carol.white@email.com', '+1-555-0103', 'active'),
        ('David Brown', 'david.brown@email.com', '+1-555-0104', 'disabled'),
        ('Eve Davis', 'eve.davis@email.com', '+1-555-0105', 'active'),
        ('Frank Miller', 'frank.miller@email.com', '+1-555-0106', 'active'),
        ('Grace Wilson', 'grace.wilson@email.com', '+1-555-0107', 'active'),
        ('Henry Moore', 'henry.moore@email.com', '+1-555-0108', 'disabled'),
        ('Iris Taylor', 'iris.taylor@email.com', '+1-555-0109', 'active'),
        ('Jack Anderson', 'jack.anderson@email.com', '+1-555-0110', 'active'),
    ]

    cursor.executemany('''
        INSERT INTO customers (name, email, phone, status)
        VALUES (?, ?, ?, ?)
    ''', sample_customers)

    conn.commit()

    # Show what we created
    cursor.execute('SELECT COUNT(*) FROM customers')
    count = cursor.fetchone()[0]

    cursor.execute('SELECT COUNT(*) FROM customers WHERE status = "active"')
    active_count = cursor.fetchone()[0]

    cursor.execute('SELECT COUNT(*) FROM customers WHERE status = "disabled"')
    disabled_count = cursor.fetchone()[0]

    conn.close()

    print(f"\n✅ Database initialized successfully!")
    print(f"📊 Total customers: {count}")
    print(f"   - Active: {active_count}")
    print(f"   - Disabled: {disabled_count}")
    print(f"💾 Database location: {DB_PATH}")

# Initialize the database
init_database()

---
## 🛠️ Customer Management Functions

Now let's create the Python functions that will perform CRUD (Create, Read, Update, Delete) operations on our customer database.

These functions will later be exposed as **MCP Tools** that AI assistants can call.

💡 **Learning Point**: Each function includes proper error handling and returns structured data. This is important for MCP tools because AI assistants need clear, consistent responses.

In [ ]:
import sqlite3
import json
from datetime import datetime
from typing import Optional, Dict, List, Any

def get_db_connection():
    """Create a database connection with row factory for dict-like access."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row  # This allows us to access columns by name
    return conn

def row_to_dict(row: sqlite3.Row) -> Dict[str, Any]:
    """Convert a SQLite row to a dictionary."""
    return {key: row[key] for key in row.keys()}

# ==================== READ OPERATIONS ====================

def get_customer(customer_id: int) -> Dict[str, Any]:
    """
    Retrieve a specific customer by ID.

    Args:
        customer_id: The unique ID of the customer

    Returns:
        Dict containing customer data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        if row:
            return {
                'success': True,
                'customer': row_to_dict(row)
            }
        else:
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

def list_customers(status: Optional[str] = None) -> Dict[str, Any]:
    """
    List all customers, optionally filtered by status.

    Args:
        status: Optional filter - 'active', 'disabled', or None for all

    Returns:
        Dict containing list of customers or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if status:
            if status not in ['active', 'disabled']:
                return {
                    'success': False,
                    'error': 'Status must be "active" or "disabled"'
                }
            cursor.execute('SELECT * FROM customers WHERE status = ? ORDER BY name', (status,))
        else:
            cursor.execute('SELECT * FROM customers ORDER BY name')

        rows = cursor.fetchall()
        conn.close()

        customers = [row_to_dict(row) for row in rows]

        return {
            'success': True,
            'count': len(customers),
            'customers': customers
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# ==================== CREATE OPERATION ====================

def add_customer(name: str, email: Optional[str] = None, phone: Optional[str] = None) -> Dict[str, Any]:
    """
    Add a new customer to the database.

    Args:
        name: Customer's full name (required)
        email: Customer's email address (optional)
        phone: Customer's phone number (optional)

    Returns:
        Dict containing the new customer data or error message
    """
    try:
        if not name or not name.strip():
            return {
                'success': False,
                'error': 'Customer name is required'
            }

        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('''
            INSERT INTO customers (name, email, phone, status)
            VALUES (?, ?, ?, 'active')
        ''', (name.strip(), email, phone))

        customer_id = cursor.lastrowid
        conn.commit()

        # Fetch the newly created customer
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        return {
            'success': True,
            'message': f'Customer created with ID {customer_id}',
            'customer': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# ==================== UPDATE OPERATIONS ====================

def update_customer(customer_id: int, name: Optional[str] = None,
                   email: Optional[str] = None, phone: Optional[str] = None) -> Dict[str, Any]:
    """
    Update customer information.

    Args:
        customer_id: The unique ID of the customer to update
        name: New name (optional)
        email: New email (optional)
        phone: New phone (optional)

    Returns:
        Dict containing updated customer data or error message
    """
    try:
        # Check if customer exists
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        # Build update query dynamically based on provided fields
        updates = []
        params = []

        if name is not None:
            updates.append('name = ?')
            params.append(name.strip())
        if email is not None:
            updates.append('email = ?')
            params.append(email)
        if phone is not None:
            updates.append('phone = ?')
            params.append(phone)

        if not updates:
            conn.close()
            return {
                'success': False,
                'error': 'No fields to update'
            }

        # Always update the updated_at timestamp
        updates.append('updated_at = CURRENT_TIMESTAMP')
        params.append(customer_id)

        update_clause = ', '.join(updates)
        query = f'UPDATE customers SET {update_clause} WHERE id = ?'
        cursor.execute(query, params)
        conn.commit()

        # Fetch updated customer
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        return {
            'success': True,
            'message': f'Customer {customer_id} updated successfully',
            'customer': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

def disable_customer(customer_id: int) -> Dict[str, Any]:
    """
    Set customer status to 'disabled'.

    Args:
        customer_id: The unique ID of the customer to disable

    Returns:
        Dict containing updated customer data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        cursor.execute('''
            UPDATE customers
            SET status = 'disabled', updated_at = CURRENT_TIMESTAMP
            WHERE id = ?
        ''', (customer_id,))
        conn.commit()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        return {
            'success': True,
            'message': f'Customer {customer_id} has been disabled',
            'customer': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

def activate_customer(customer_id: int) -> Dict[str, Any]:
    """
    Set customer status to 'active'.

    Args:
        customer_id: The unique ID of the customer to activate

    Returns:
        Dict containing updated customer data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        cursor.execute('''
            UPDATE customers
            SET status = 'active', updated_at = CURRENT_TIMESTAMP
            WHERE id = ?
        ''', (customer_id,))
        conn.commit()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        return {
            'success': True,
            'message': f'Customer {customer_id} has been activated',
            'customer': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# Test the functions
print("✅ Customer management functions defined successfully!")
print("\n📋 Available functions:")
print("   - get_customer(customer_id)")
print("   - list_customers(status=None)")
print("   - add_customer(name, email=None, phone=None)")
print("   - update_customer(customer_id, name=None, email=None, phone=None)")
print("   - disable_customer(customer_id)")
print("   - activate_customer(customer_id)")

# Quick test
print("\n🧪 Quick test - Fetching customer ID 1:")
result = get_customer(1)
if result['success']:
    customer = result['customer']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")

---
## 🌐 MCP HTTP Streaming Server Implementation

Now for the exciting part! We'll build an MCP server that:
1. Implements the MCP protocol specification
2. Uses Server-Sent Events (SSE) for streaming responses
3. Exposes our customer management functions as MCP tools
4. Runs in a background thread so Colab remains responsive

### MCP Protocol Overview:

The MCP protocol uses JSON-RPC 2.0 messages over HTTP with SSE. Key message types:
- **initialize**: Handshake to establish connection and capabilities
- **tools/list**: Request list of available tools
- **tools/call**: Execute a specific tool

💡 **Learning Point**: SSE (Server-Sent Events) allows the server to push updates to the client. Each message starts with `data: ` and ends with `\n\n`.

In [ ]:
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
import json
import threading
import time
from typing import Dict, Any, Generator

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Server state
server_thread = None
server_running = False

# MCP Protocol Implementation

# Define the tools that will be exposed via MCP
MCP_TOOLS = [
    {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, and status.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "list_customers",
        "description": "List all customers in the database. Can optionally filter by status (active or disabled).",
        "inputSchema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "Optional filter by customer status"
                }
            }
        }
    },
    {
        "name": "add_customer",
        "description": "Add a new customer to the database. Name is required, email and phone are optional.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Customer's full name (required)"
                },
                "email": {
                    "type": "string",
                    "description": "Customer's email address (optional)"
                },
                "phone": {
                    "type": "string",
                    "description": "Customer's phone number (optional)"
                }
            },
            "required": ["name"]
        }
    },
    {
        "name": "update_customer",
        "description": "Update an existing customer's information. Provide the customer ID and the fields to update.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to update"
                },
                "name": {
                    "type": "string",
                    "description": "New name (optional)"
                },
                "email": {
                    "type": "string",
                    "description": "New email (optional)"
                },
                "phone": {
                    "type": "string",
                    "description": "New phone (optional)"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "disable_customer",
        "description": "Disable a customer account by setting their status to 'disabled'.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to disable"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "activate_customer",
        "description": "Activate a customer account by setting their status to 'active'.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to activate"
                }
            },
            "required": ["customer_id"]
        }
    }
]

def create_sse_message(data: Dict[str, Any]) -> str:
    """
    Format a message for Server-Sent Events (SSE).
    SSE format: 'data: {json}\n\n'
    """
    return f"data: {json.dumps(data)}\n\n"

def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle MCP initialize request.
    This is the first message in the MCP protocol handshake.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "tools": {},  # We support tools
            },
            "serverInfo": {
                "name": "customer-management-server",
                "version": "1.0.0"
            }
        }
    }

def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/list request.
    Returns the list of available tools.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "tools": MCP_TOOLS
        }
    }

def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/call request.
    Executes the requested tool and returns the result.
    """
    params = message.get("params", {})
    tool_name = params.get("name")
    arguments = params.get("arguments", {})

    # Map tool names to functions
    tool_functions = {
        "get_customer": get_customer,
        "list_customers": list_customers,
        "add_customer": add_customer,
        "update_customer": update_customer,
        "disable_customer": disable_customer,
        "activate_customer": activate_customer,
    }

    if tool_name not in tool_functions:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Tool not found: {tool_name}"
            }
        }

    try:
        # Call the tool function with the provided arguments
        result = tool_functions[tool_name](**arguments)

        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": json.dumps(result, indent=2)
                    }
                ]
            }
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32603,
                "message": f"Tool execution error: {str(e)}"
            }
        }

def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process an MCP message and route it to the appropriate handler.
    """
    method = message.get("method")

    if method == "initialize":
        return handle_initialize(message)
    elif method == "tools/list":
        return handle_tools_list(message)
    elif method == "tools/call":
        return handle_tools_call(message)
    else:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Method not found: {method}"
            }
        }

# Flask Routes

@app.route('/mcp', methods=['POST'])
def mcp_endpoint():
    """
    Main MCP endpoint for MCP communication.
    Receives MCP messages and streams responses using Server-Sent Events.
    """
    # Get the MCP message from the request BEFORE entering the generator
    # This must be done in the request context
    message = request.get_json()

    def generate():
        try:
            print(f"📥 Received MCP message: {message.get('method')}")

            # Process the message
            response = process_mcp_message(message)

            print(f"📤 Sending MCP response")

            # Send the response as SSE
            yield create_sse_message(response)

        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": None,
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)

    return Response(generate(), mimetype='text/event-stream')

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint to verify server is running."""
    return jsonify({
        "status": "healthy",
        "server": "customer-management-mcp-server",
        "version": "1.0.0"
    })

print("✅ MCP Server implementation complete!")
print("\n🔧 Server features:")
print("   - MCP protocol support (2024-11-05)")
print("   - Server-Sent Events (SSE) streaming")
print(f"   - {len(MCP_TOOLS)} tools exposed")
print("   - Health check endpoint")
print("   - CORS enabled for cross-origin requests")

---
## 🚀 Start the MCP Server

Now let's start the server in a background thread so it runs alongside our Colab notebook.

### Features:
- **Background Threading**: Server runs without blocking the notebook
- **Local Access**: Test directly from this notebook
- **Public Access**: Automatic ngrok tunnel for external testing
- **MCP Inspector**: Test with Anthropic's MCP Inspector tool

💡 **Learning Point**: We use threading to run the Flask server in the background. This allows the Colab notebook to remain interactive while the server handles requests.

🌐 **Ngrok Tunnel**: By default, the server creates a public URL using ngrok. This allows you to test the MCP server from anywhere, including with the [MCP Inspector](https://inspector.anthropic.com)!## 🚀 Start the MCP Server

Now let's start the server in a background thread so it runs alongside our Colab notebook.

💡 **Learning Point**: We use threading to run the Flask server in the background. This allows the Colab notebook to remain interactive while the server handles requests.

In [ ]:
import threading
import time
import requests
from termcolor import colored
from pyngrok import ngrok
from google.colab import userdata

# Server configuration
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'

def run_server():
    """Run the Flask server in a separate thread."""
    global server_running
    server_running = True
    app.run(host=SERVER_HOST, port=SERVER_PORT, debug=False, use_reloader=False)

def start_server(use_ngrok=True):
    """Start the MCP server in a background thread."""
    global server_thread, server_running

    if server_thread and server_thread.is_alive():
        print(colored("⚠️  Server is already running!", "yellow"))
        return

    print(colored("🚀 Starting MCP server...", "cyan"))

    # Start server in background thread
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Wait for server to start
    time.sleep(2)

    # Check if server is healthy
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=5)
        if response.status_code == 200:
            print(colored("✅ MCP Server is running!", "green"))
            print(colored(f"📍 Local URL: {SERVER_URL}", "cyan"))

            # Set up ngrok tunnel if requested
            if use_ngrok:
                print(colored("\n🌐 Setting up public tunnel with ngrok...", "cyan"))
                try:
                    # Get ngrok authtoken from Colab secrets
                    try:
                        authtoken = userdata.get('NGROK_AUTHTOKEN')
                        ngrok.set_auth_token(authtoken)
                        print(colored("✅ Ngrok authenticated", "green"))
                    except Exception as e:
                        print(colored("⚠️  NGROK_AUTHTOKEN not found in Colab secrets", "yellow"))
                        print(colored("   To use ngrok:", "yellow"))
                        print(colored("   1. Get free authtoken from https://ngrok.com", "yellow"))
                        print(colored("   2. In Colab: Click 🔑 (Secrets) in left sidebar", "yellow"))
                        print(colored("   3. Add secret: Name='NGROK_AUTHTOKEN', Value=<your-token>", "yellow"))
                        print(colored("   4. Enable 'Notebook access' for the secret", "yellow"))
                        print(colored("   5. Re-run this cell", "yellow"))
                        print(colored("\n   Server is still accessible locally at " + SERVER_URL, "cyan"))
                        return

                    # Create ngrok tunnel
                    public_url = ngrok.connect(SERVER_PORT)
                    print(colored(f"✅ Public URL: {public_url}", "green", attrs=["bold"]))
                    print(colored(f"📍 MCP Endpoint: {public_url}/mcp", "green", attrs=["bold"]))
                    print(colored(f"📍 Health Check: {public_url}/health", "cyan"))
                    print()
                    print(colored("🔍 MCP Inspector Instructions:", "yellow", attrs=["bold"]))
                    print(colored("1. Run in terminal: npx @modelcontextprotocol/inspector", "yellow"))
                    print(colored("2. This will open MCP Inspector in your browser", "yellow"))
                    print(colored(f"3. Enter MCP URL: {public_url}/mcp", "yellow"))
                    print(colored("4. Click 'Connect' and test the customer management tools!", "yellow"))
                except Exception as e:
                    if "NGROK_AUTHTOKEN" not in str(e):
                        print(colored(f"⚠️  Could not set up ngrok tunnel: {e}", "yellow"))
                        print(colored("   Server is still accessible locally", "yellow"))
        else:
            print(colored("❌ Server started but health check failed", "red"))
    except Exception as e:
        print(colored(f"❌ Failed to connect to server: {e}", "red"))

def stop_server():
    """Stop the MCP server."""
    global server_running
    server_running = False
    print(colored("🛑 Server stopped", "yellow"))
    print(colored("   Note: In Colab, the thread will continue until the runtime is reset", "yellow"))

def check_server_status():
    """Check if the server is running."""
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=2)
        if response.status_code == 200:
            print(colored("✅ Server is running and healthy", "green"))
            health_data = response.json()
            print(f"   Status: {health_data['status']}")
            print(f"   Server: {health_data['server']}")
            print(f"   Version: {health_data['version']}")
            return True
        else:
            print(colored("❌ Server is not responding correctly", "red"))
            return False
    except Exception as e:
        print(colored("❌ Server is not running", "red"))
        print(f"   Error: {e}")
        return False

# Start the server
start_server()


---
## 🧪 Test Section: MCP Protocol in Action

Now let's test our MCP server by sending actual MCP protocol messages!

We'll demonstrate:
1. **Initialize** - Establish connection
2. **List Tools** - Discover available tools
3. **Call Tools** - Execute customer management operations

💡 **Learning Point**: Watch how MCP messages are structured using JSON-RPC 2.0 format. Each request has an `id`, `method`, and optional `params`.

### Test 1: Initialize Connection

The first step in MCP is always initialization. This establishes the protocol version and capabilities.

In [ ]:
import requests
import json
from termcolor import colored

def send_mcp_message(method: str, params: dict = None, message_id: int = 1):
    """
    Send an MCP message to the server and display the request/response.
    """
    # Construct MCP message
    message = {
        "jsonrpc": "2.0",
        "id": message_id,
        "method": method
    }

    if params:
        message["params"] = params

    print(colored(f"\n📤 Sending MCP Request:", "cyan", attrs=["bold"]))
    print(colored(json.dumps(message, indent=2), "cyan"))

    try:
        # Send request to MCP endpoint
        response = requests.post(
            f'{SERVER_URL}/mcp',
            json=message,
            headers={'Content-Type': 'application/json'},
            stream=True,
            timeout=10
        )

        # Parse SSE response
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8')
                if line_str.startswith('data: '):
                    data = json.loads(line_str[6:])  # Remove 'data: ' prefix

                    print(colored(f"\n📥 Received MCP Response:", "green", attrs=["bold"]))
                    print(colored(json.dumps(data, indent=2), "green"))

                    return data

    except Exception as e:
        print(colored(f"\n❌ Error: {e}", "red"))
        return None

# Test 1: Initialize
print(colored("="*60, "magenta"))
print(colored("TEST 1: MCP INITIALIZATION", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

init_response = send_mcp_message(
    method="initialize",
    params={
        "protocolVersion": "2024-11-05",
        "capabilities": {},
        "clientInfo": {
            "name": "colab-test-client",
            "version": "1.0.0"
        }
    },
    message_id=1
)

if init_response and 'result' in init_response:
    print(colored("\n✅ Initialization successful!", "green", attrs=["bold"]))
    print(f"   Protocol Version: {init_response['result']['protocolVersion']}")
    print(f"   Server: {init_response['result']['serverInfo']['name']}")
else:
    print(colored("\n❌ Initialization failed", "red", attrs=["bold"]))

### Test 2: List Available Tools

After initialization, let's discover what tools the server offers.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 2: LIST AVAILABLE TOOLS", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

tools_response = send_mcp_message(
    method="tools/list",
    message_id=2
)

if tools_response and 'result' in tools_response:
    tools = tools_response['result']['tools']
    print(colored(f"\n✅ Found {len(tools)} tools:", "green", attrs=["bold"]))
    for i, tool in enumerate(tools, 1):
        print(colored(f"\n{i}. {tool['name']}", "yellow", attrs=["bold"]))
        print(f"   {tool['description']}")
else:
    print(colored("\n❌ Failed to list tools", "red", attrs=["bold"]))

### Test 3: Call Tool - List All Customers

Let's call our first tool to list all customers in the database.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 3: LIST ALL CUSTOMERS", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

list_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "list_customers",
        "arguments": {}
    },
    message_id=3
)

if list_response and 'result' in list_response:
    content = list_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        print(colored(f"\n✅ Found {data['count']} customers:", "green", attrs=["bold"]))
        for customer in data['customers']:
            status_color = "green" if customer['status'] == 'active' else "red"
            print(f"\n   ID: {customer['id']}")
            print(f"   Name: {customer['name']}")
            print(f"   Email: {customer['email']}")
            print(f"   Phone: {customer['phone']}")
            print(colored(f"   Status: {customer['status']}", status_color))
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 4: Call Tool - Get Specific Customer

Retrieve details for a specific customer by ID.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 4: GET CUSTOMER BY ID", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

get_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "get_customer",
        "arguments": {
            "customer_id": 1
        }
    },
    message_id=4
)

if get_response and 'result' in get_response:
    content = get_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored("\n✅ Customer found:", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(f"   Email: {customer['email']}")
        print(f"   Phone: {customer['phone']}")
        print(f"   Status: {customer['status']}")
        print(f"   Created: {customer['created_at']}")
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 5: Call Tool - Add New Customer

Create a new customer in the database.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 5: ADD NEW CUSTOMER", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

add_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "add_customer",
        "arguments": {
            "name": "Sarah Connor",
            "email": "sarah.connor@resistance.com",
            "phone": "+1-555-TERMINATE"
        }
    },
    message_id=5
)

if add_response and 'result' in add_response:
    content = add_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored(f"\n✅ {data['message']}", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(f"   Email: {customer['email']}")
        print(f"   Phone: {customer['phone']}")
        print(f"   Status: {customer['status']}")
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 6: Call Tool - Update Customer

Update an existing customer's information.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 6: UPDATE CUSTOMER INFORMATION", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

update_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "update_customer",
        "arguments": {
            "customer_id": 2,
            "email": "bob.smith.updated@email.com",
            "phone": "+1-555-9999"
        }
    },
    message_id=6
)

if update_response and 'result' in update_response:
    content = update_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored(f"\n✅ {data['message']}", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(colored(f"   Email: {customer['email']} (updated)", "yellow"))
        print(colored(f"   Phone: {customer['phone']} (updated)", "yellow"))
        print(f"   Updated at: {customer['updated_at']}")
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 7: Call Tool - Disable Customer

Disable a customer account.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 7: DISABLE CUSTOMER ACCOUNT", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

disable_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "disable_customer",
        "arguments": {
            "customer_id": 5
        }
    },
    message_id=7
)

if disable_response and 'result' in disable_response:
    content = disable_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored(f"\n✅ {data['message']}", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(colored(f"   Status: {customer['status']} (changed)", "red"))
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 8: Call Tool - Activate Customer

Re-activate a disabled customer account.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 8: ACTIVATE CUSTOMER ACCOUNT", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

activate_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "activate_customer",
        "arguments": {
            "customer_id": 4
        }
    },
    message_id=8
)

if activate_response and 'result' in activate_response:
    content = activate_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        customer = data['customer']
        print(colored(f"\n✅ {data['message']}", "green", attrs=["bold"]))
        print(f"   ID: {customer['id']}")
        print(f"   Name: {customer['name']}")
        print(colored(f"   Status: {customer['status']} (changed)", "green"))
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 9: Error Handling - Invalid Customer ID

Let's test how the server handles errors, like requesting a non-existent customer.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 9: ERROR HANDLING - NON-EXISTENT CUSTOMER", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

error_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "get_customer",
        "arguments": {
            "customer_id": 99999
        }
    },
    message_id=9
)

if error_response and 'result' in error_response:
    content = error_response['result']['content'][0]['text']
    data = json.loads(content)

    print(colored("\n💡 Error handling demonstration:", "yellow", attrs=["bold"]))
    print(f"   Success: {data['success']}")
    print(colored(f"   Error Message: {data['error']}", "yellow"))
    print(colored("\n✅ The server correctly handled the invalid request!", "green"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

### Test 10: List Active Customers Only

Demonstrate filtering capabilities.

In [ ]:
print(colored("="*60, "magenta"))
print(colored("TEST 10: LIST ACTIVE CUSTOMERS ONLY", "magenta", attrs=["bold"]))
print(colored("="*60, "magenta"))

active_response = send_mcp_message(
    method="tools/call",
    params={
        "name": "list_customers",
        "arguments": {
            "status": "active"
        }
    },
    message_id=10
)

if active_response and 'result' in active_response:
    content = active_response['result']['content'][0]['text']
    data = json.loads(content)

    if data['success']:
        print(colored(f"\n✅ Found {data['count']} active customers:", "green", attrs=["bold"]))
        for customer in data['customers']:
            print(f"   - {customer['name']} ({customer['email']})")
    else:
        print(colored(f"\n❌ Error: {data['error']}", "red"))
else:
    print(colored("\n❌ Tool call failed", "red", attrs=["bold"]))

---
## 🔍 Testing with MCP Inspector

Now that you've tested the server locally, let's try it with **MCP Inspector**!

### What is MCP Inspector?

MCP Inspector is a web-based tool from Anthropic that provides a visual interface for testing MCP servers. It's like Postman, but specifically designed for the MCP protocol.

### How to Use MCP Inspector:

1. **Get your public URL** from the server startup output above (it looks like `https://xxxx.ngrok.io`)

2. **Open MCP Inspector**:
   - Go to [https://inspector.anthropic.com](https://inspector.anthropic.com)

3. **Connect to your server**:
   - In the "Server URL" field, enter: `<your-ngrok-url>/mcp`
   - Example: `https://abc123.ngrok.io/mcp`
   - Click "Connect"

4. **Explore the tools**:
   - You'll see all 6 customer management tools listed
   - Click on any tool to see its description and parameters

5. **Test a tool**:
   - Click on "list_customers"
   - Leave the parameters empty (or add `{"status": "active"}`)
   - Click "Call Tool"
   - See the results in real-time!

6. **Try other tools**:
   - Get a specific customer: `{"customer_id": 1}`
   - Add a customer: `{"name": "John Doe", "email": "john@example.com"}`
   - Update, disable, or activate customers

### Benefits of MCP Inspector:

- ✅ Visual interface for testing
- ✅ No coding required
- ✅ See the full MCP protocol messages
- ✅ Debug issues easily
- ✅ Share the URL with others for collaborative testing

💡 **Learning Point**: MCP Inspector is invaluable for debugging and demonstrating MCP servers. It shows you exactly what messages are being sent and received, helping you understand the protocol better.

Try it now with your server!

---
## 🎯 Server Management

Utility cells for managing the MCP server.

In [ ]:
# Check server status
check_server_status()

In [ ]:
# Stop the server (note: thread will continue until runtime reset in Colab)
stop_server()

---
## 📝 Summary and Learning Points

### What We Built

Congratulations! You've successfully built and tested a complete MCP server. Here's what we created:

1. **SQLite Database** - A persistent storage system for customer data
2. **CRUD Functions** - Six Python functions for managing customer records
3. **MCP Server** - A Flask-based HTTP server implementing the MCP protocol
4. **SSE Streaming** - Server-Sent Events for real-time communication
5. **Six MCP Tools** - Customer management capabilities exposed via MCP

### Key MCP Concepts Learned

#### 1. MCP Protocol Structure
```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": { ... }
}
```

#### 2. Tool Definition Schema
Every tool needs:
- **name**: Unique identifier
- **description**: What the tool does
- **inputSchema**: JSON Schema defining parameters

#### 3. Server-Sent Events (SSE)
```
data: {"jsonrpc":"2.0", ...}

```

#### 4. Three Main MCP Methods
- **initialize**: Handshake and capability negotiation
- **tools/list**: Discovery of available tools
- **tools/call**: Execute a specific tool

### Real-World Applications

This pattern can be extended to:
- **CRM Systems**: Customer relationship management
- **Inventory Management**: Product and warehouse tracking
- **API Integration**: Connect AI to external services
- **Data Analysis**: Let AI query and analyze databases
- **Automation**: AI-driven business process automation

### Next Steps

To extend this demo:
1. **Add Authentication**: Implement API keys or OAuth
2. **Add More Tools**: Create, delete, search with filters
3. **Add Resources**: Expose database records as MCP resources
4. **Add Prompts**: Create prompt templates for common queries
5. **Deploy**: Move from Colab to a production server
6. **Connect Claude**: Use Claude Desktop or API to interact with your server

### Resources

- **MCP Specification**: [modelcontextprotocol.io](https://modelcontextprotocol.io)
- **MCP Python SDK**: [github.com/modelcontextprotocol/python-sdk](https://github.com/modelcontextprotocol/python-sdk)
- **Flask Documentation**: [flask.palletsprojects.com](https://flask.palletsprojects.com)
- **SQLite Tutorial**: [sqlite.org/docs.html](https://sqlite.org/docs.html)

---

## 🎓 Educational Exercise

Try these challenges to deepen your understanding:

1. **Add a Search Tool**: Create a `search_customers` tool that searches by name or email
2. **Add Validation**: Enhance input validation for email and phone formats
3. **Add Logging**: Track all MCP calls to a log file
4. **Add Statistics**: Create a tool that returns customer statistics
5. **Add Pagination**: Modify `list_customers` to support pagination

---

### 💡 Final Thoughts

MCP is a powerful protocol that bridges the gap between AI assistants and real-world data and tools. By understanding how to build MCP servers, you can:

- Extend AI capabilities with custom tools
- Integrate AI into existing systems
- Build secure, controlled interfaces for AI interactions
- Create specialized AI assistants for specific domains

The customer management system we built here is just the beginning. The same patterns apply to any domain where you want AI to interact with structured data or execute specific operations.

**Happy coding!** 🚀